## Interpretability

In [ ]:
# stdlib
import copy
import warnings
from pathlib import Path

# third party
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# autoprognosis absolute
from autoprognosis.plugins.explainers import Explainers
from autoprognosis.plugins.prediction import Predictions
from autoprognosis.utils.serialization import load_model_from_file, save_model_to_file
from autoprognosis.utils.tester import evaluate_estimator

warnings.filterwarnings("ignore")


models_path = Path("./workspace")
print(models_path.absolute())
models_path.mkdir(parents=True, exist_ok=True)

In [ ]:
Explainers().list_available()

In [ ]:
Predictions().list_available()

In [ ]:
models = {
    "lime": {},
    "kernel_shap": {},
    "invase": {},
}

## Dataset

In [ ]:
# stdlib
import copy

# third party
import pandas as pd
from sklearn.model_selection import train_test_split

# Wine Quality Data Set
df = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
    sep=";",
)


last_col = df.columns[-1]

y = df[last_col]
mapped_labels = sorted(y.unique())
mapping = {}
for idx, label in enumerate(mapped_labels):
    mapping[label] = idx
y = y.map(mapping)

X = df.drop(columns=[last_col])

X_train, X_test, y_train, y_test = train_test_split(X, y)

test_set = X_test[:10]
columns = X.columns

X

## XGBoost

In [ ]:
# Test sanity

model = Predictions(category="classifier").get("xgboost")

evaluate_estimator(model, X_train, y_train, n_folds=2)["str"]

In [ ]:
# lime


def get_lime(category, model_name):
    model = Predictions(category).get(model_name)
    model_file = models_path / "lime.p"

    if not model_file.exists():
        exp = Explainers().get(
            "lime",
            copy.deepcopy(model),
            X_train,
            y_train,
            feature_names=columns,
            task_type="classification",
        )
        save_model_to_file(model_file, exp)
    else:
        exp = load_model_from_file(model_file)

    return exp


def explain_lime(test_set, category, model_name):
    exp = get_lime(category, model_name)
    output = exp.explain(test_set).squeeze()
    print("importance \n", output)
    exp.plot(output, feature_names=list(output.index))


explain_lime(test_set.head(1), "classifier", "xgboost")

In [ ]:
# kernel_shap
def get_kernel_shap(category, model_name):
    model = Predictions(category).get(model_name)
    model_file = models_path / "kernel_shap.p"

    if not model_file.exists():
        exp = Explainers().get(
            "kernel_shap",
            copy.deepcopy(model),
            X_train,
            y_train,
            feature_names=columns,
            task_type="classification",
        )
        save_model_to_file(model_file, exp)
    else:
        exp = load_model_from_file(model_file)

    return exp


def explain_kernel_shap(test_set, category, model_name):
    exp = get_kernel_shap(category, model_name)
    exp.plot(test_set)


explain_kernel_shap(test_set, "classifier", "xgboost")

In [ ]:
# invase
def get_invase(category, model_name):
    model = Predictions(category).get(model_name)
    model_file = models_path / "invase.p"

    if not model_file.exists():
        exp = Explainers().get(
            "invase",
            copy.deepcopy(model),
            X_train,
            y_train,
            feature_names=columns,
            n_folds=1,
            n_epoch=200,
            task_type="classification",
        )
        save_model_to_file(model_file, exp)
    else:
        exp = load_model_from_file(model_file)

    return exp


def explain_invase(test_set, category, model_name):
    exp = get_invase(category, model_name)
    output = exp.explain(test_set)

    exp.plot(output)


explain_invase(test_set, "classifier", "xgboost")

# Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star AutoPrognosis on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.

- [Star AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
- [Star HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
